In [13]:
import sqlite3
import os
import pandas as pd

# --- Configuration ---
# Path to your downloaded Chinook database file.
# This is now set to the path you provided.
DATABASE_FILE = 'testing_data/chinook.db'

def table_to_dataframe(db_file, table_name):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        query = f"SELECT * FROM {table_name}".upper()
        df = pd.read_sql_query(query, conn)
        
        return df
    
    except Exception as e:
        return None
    
    finally:
        if conn:
            conn.close()


def list_tables(db_file):
    """
    Lists all tables in the specified SQLite database file.

    Args:
        db_file (str): The path to the SQLite database file.

    Returns:
        list: A list of table names, or None if an error occurs.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        # Execute a query to fetch the names of all tables
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()
        # The result is a list of tuples, so we extract the first element of each tuple
        return [table[0] for table in tables]
    except sqlite3.Error as e:
        print(f"An error occurred: {e}")
        return None
    finally:
        if conn:
            conn.close()



def list_table_features(db_file, table_name):
    """
    Lists all features (columns) and their data types for a given table.

    Args:
        db_file (str): The path to the SQLite database file.
        table_name (str): The name of the table.

    Returns:
        list: A list of tuples, where each tuple contains (column_name, data_type),
              or None if an error occurs or the table doesn't exist.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        # PRAGMA table_info is a special command to get table structure
        # It's generally safe from SQL injection for table_name if table_name is validated
        # or comes from a trusted source (like output of list_tables).
        cursor.execute(f"PRAGMA table_info('{table_name}');")
        columns_info = cursor.fetchall()
        if not columns_info:
            print(f"Table '{table_name}' not found or has no columns.")
            return None
        # Each row from PRAGMA table_info contains:
        # (cid, name, type, notnull, dflt_value, pk)
        # We are interested in name (index 1) and type (index 2)
        return [info[1] for info in columns_info]
    except sqlite3.Error as e:
        print(f"An error occurred with list_table_features for table '{table_name}': {e}")
        return None
    except Exception as e:
        print(f"A general error occurred in list_table_features: {e}")
        return None
    finally:
        if conn:
            conn.close()

def get_table_schema(db_file, table_name):
    """
    Retrieves the schema of a given table and returns it as a pandas DataFrame.

    Args:
        db_file (str): The path to the SQLite database file.
        table_name (str): The name of the table.

    Returns:
        pandas.DataFrame: A DataFrame containing the schema (column ID, name, type,
                          not null, default value, primary key), or None if an error occurs.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        # Using PRAGMA table_info which is standard for SQLite
        query = f"PRAGMA table_info('{table_name}');"
        df = pd.read_sql_query(query, conn)
        if df.empty:
            print(f"Table '{table_name}' not found or is empty.")
            return None
        return df
    except sqlite3.Error as e:
        print(f"An SQLite error occurred in get_table_schema_df for table '{table_name}': {e}")
        return None
    except Exception as e:
        print(f"A general error occurred in get_table_schema_df: {e}")
        return None
    finally:
        if conn:
            conn.close()

def query_to_dataframe(db_file, query, params=None):
    """
    Executes a custom SELECT SQL query and returns the results as a pandas DataFrame.

    Args:
        db_file (str): The path to the SQLite database file.
        query (str): The SELECT SQL query to execute.
        params (tuple, optional): Parameters to substitute into the query for safety. Defaults to None.

    Returns:
        pandas.DataFrame: A DataFrame containing the query results, or None if an error occurs.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        if params:
            df = pd.read_sql_query(query, conn, params=params)
        else:
            df = pd.read_sql_query(query, conn)
        return df
    except sqlite3.Error as e:
        print(f"An SQLite error occurred in fetch_query_to_dataframe: {e}")
        return None
    except Exception as e: # Catch pandas related errors too
        print(f"A general error occurred in fetch_query_to_dataframe: {e}")
        return None
    finally:
        if conn:
            conn.close()

def count_table_rows(db_file, table_name):
    """
    Counts the number of rows in a specific table.

    Args:
        db_file (str): The path to the SQLite database file.
        table_name (str): The name of the table.

    Returns:
        int: The number of rows in the table, or None if an error occurs.
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        cursor = conn.cursor()
        # Sanitize table_name or ensure it's from a trusted source
        # For this function, using an f-string is common, but be cautious
        cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
        count = cursor.fetchone()[0]
        return count
    except sqlite3.Error as e:
        print(f"An error occurred with count_table_rows for table '{table_name}': {e}")
        return None
    finally:
        if conn:
            conn.close()

def get_all_table_features(db_file):
    """
    Retrieves all tables in the database and their respective features (columns and types).

    Args:
        db_file (str): The path to the SQLite database file.

    Returns:
        dict: A dictionary where keys are table names (str) and values are lists
              of tuples, each tuple containing (column_name, data_type).
              Returns None if listing tables fails, or an empty dict if no tables exist.
              Individual tables might have a value of None if fetching their features failed.
    """
    all_tables = list_tables(db_file)
    if all_tables is None:
        print("Failed to retrieve list of tables.")
        return None

    if not all_tables:
        print("No tables found in the database.")
        return {}

    database_features_dict = {}
    for table_name in all_tables:
        features = list_table_features(db_file, table_name)
        # list_table_features returns a list of (name, type) tuples,
        # or None on error, or an empty list if table has no columns/not found by PRAGMA.
        database_features_dict[table_name] = features
        if features is None:
            print(f"Could not retrieve features for table '{table_name}'. It will have a None value in the dictionary.")
        elif not features: # Empty list
             print(f"Table '{table_name}' has no features listed or could not be found by PRAGMA table_info.")


    return database_features_dict

In [11]:
df1= table_to_dataframe(DATABASE_FILE, "artists")
df1

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
270,271,"Mela Tenenbaum, Pro Musica Prague & Richard Kapp"
271,272,Emerson String Quartet
272,273,"C. Monteverdi, Nigel Rogers - Chiaroscuro; Lon..."
273,274,Nash Ensemble


In [12]:
params = {
    "db_file": DATABASE_FILE,
    "table_name": "invoices"
}

list_table_features(**params)

['InvoiceId',
 'CustomerId',
 'InvoiceDate',
 'BillingAddress',
 'BillingCity',
 'BillingState',
 'BillingCountry',
 'BillingPostalCode',
 'Total']

In [10]:
list_tables(DATABASE_FILE)


['albums',
 'sqlite_sequence',
 'artists',
 'customers',
 'employees',
 'genres',
 'invoices',
 'invoice_items',
 'media_types',
 'playlists',
 'playlist_track',
 'tracks',
 'sqlite_stat1']

In [15]:
features = get_all_table_features(DATABASE_FILE)

In [16]:
for table in features:
    print(table)

albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists
playlist_track
tracks
sqlite_stat1


In [17]:
features['employees']

['EmployeeId',
 'LastName',
 'FirstName',
 'Title',
 'ReportsTo',
 'BirthDate',
 'HireDate',
 'Address',
 'City',
 'State',
 'Country',
 'PostalCode',
 'Phone',
 'Fax',
 'Email']

In [21]:
len(str(features)+"hithere")

1007

In [27]:
string = """
def get_features_from_all_tables(db_file):

    # Retrieves all tables in the database and their respective features (columns and types).

    # Args:
    #     db_file (str): The path to the SQLite database file.

    # Returns:
    #     dict: A dictionary where keys are table names (str) and values are lists
    #           of tuples, each tuple containing (column_name, data_type).
    #           Returns None if listing tables fails, or an empty dict if no tables exist.
    #           Individual tables might have a value of None if fetching their features failed.

    all_tables = list_tables(db_file)
    if all_tables is None:
        print("Failed to retrieve list of tables.")
        return None

    if not all_tables:
        print("No tables found in the database.")
        return {}

    database_features_dict = {}
    for table_name in all_tables:
        features = list_table_features(db_file, table_name)
        # list_table_features returns a list of (name, type) tuples,
        # or None on error, or an empty list if table has no columns/not found by PRAGMA.
        database_features_dict[table_name] = features
        if features is None:
            print(f"Could not retrieve features for table '{table_name}'. It will have a None value in the dictionary.")
        elif not features: # Empty list
             print(f"Table '{table_name}' has no features listed or could not be found by PRAGMA table_info.")


    return database_features_dict
features_and_tables = get_features_from_all_tables(DATABASE_FILE)
"""

local_var = {"DATABASE_FILE": DATABASE_FILE}

exec(string, {}, local_var)



NameError: name 'list_tables' is not defined